<a href="https://colab.research.google.com/github/squeeko/DL_TF20_KerasCNNGANSRNNNLP/blob/main/TF_GANs_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST using GAN in TensorFlow

Let us build a simple GAN capable of generating handwritten digits. We will use
the MNIST handwritten digits to train the network. We use the TensorFlow Keras
dataset to access the MNIST data. The data contains 60,000 training images of
handwritten digits each of size 28 × 28. The pixel value of the digits lies between
0-255; we normalize the input values such that each pixel has a value in range [-1, 1]:

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist

In [3]:
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

11493376/11490434 [==============================] - 0s 0us/step


We will use a simple multi-layered perceptron (MLP) and we will feed it an image
as a flat vector of size 784, so we reshape the training data:

A multilayer perceptron (MLP) is a class of feedforward artificial neural network (ANN). The term MLP is used ambiguously, sometimes loosely to any feedforward ANN, sometimes strictly to refer to networks composed of multiple layers of perceptrons (with threshold activation); see § Terminology. Multilayer perceptrons are sometimes colloquially referred to as "vanilla" neural networks, especially when they have a single hidden layer.

An MLP consists of at least three layers of nodes: an input layer, a hidden layer and an output layer. Except for the input nodes, each node is a neuron that uses a nonlinear activation function. MLP utilizes a supervised learning technique called backpropagation for training. Its multiple layers and non-linear activation distinguish MLP from a linear perceptron. It can distinguish data that is not linearly separable.

In [4]:
X_train = X_train.reshape(60000, 784)

Now we will need to build a generator and discriminator. The purpose of the
generator is to take in a noisy input and generate an image similar to the training
dataset. **The size of the noisy input is decided by the variable randomDim; you
can initialize it to any integral value. Conventionally people set it to 100. For our
implementation we tried a value of 10.** This input is fed to a Dense layer with
256 neurons with LeakyReLU activation.  We next add another Dense layer with
512 hidden neurons, followed by the third hidden layer with 1024 neurons and
finally the output layer with 784 neurons. You can change the number of neurons
in the hidden layers and see how the performance changes; however, the number
of neurons in the output unit has to match the number of pixels in the training
images. The corresponding generator is then:

In [20]:
randomDim = 10

generator = Sequential()
generator.add(Dense(256, input_dim = randomDim))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))

Similarly, we build a discriminator. Notice now that the discriminator takes in the
images, either from the training set or images generated by generator, thus its input
size is 784. The output of the discriminator however is a single bit, with 0 signifying
a fake image (generated by generator) and 1 signifying that the image is from the
training dataset:

In [18]:
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim = 784))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))

Next, we combine the generator and discriminator together to form a GAN. In the
GAN we ensure that the discriminator weights are fixed by setting the trainable
argument to False:

In [22]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim, ))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs = ganInput, outputs = ganOutput)

The trick to train the two is that we first train the discriminator separately; we
use binary cross entropy loss for the discriminator. Later we freeze the weights of
the discriminator and train the combined GAN; this results in the training of the
generator. The loss this time is also binary cross entropy:

In [33]:
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'adam')
gan.compile(loss = 'binary_crossentropy', optimizer = 'adam')

Let us now perform the training. For each epoch we take a sample of random noise
first, feed it to the generator, and the generator produces a fake image. We combine
the generated fake images and the actual training images in a batch with their
specific labels and use them to train the discriminator first on the given batch:

In [39]:
def train(epochs=1, batchSize=128):
  batchCount = int(X_train.shape[0] / batchSize)
  print ('Epochs:', epochs)
  print ('Batch size:', batchSize)
  print ('Batches per epoch:', batchCount)

  for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in range(batchCount):

    # Get a random set of input noise and images
      noise = np.random.normal(0, 1, size=[batchSize, randomDim])
      imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

      # Generate fake MNIST images
      generatedImages = generator.predict(noise)

      # print np.shape(imageBatch), np.shape(generatedImages)
      X = np.concatenate([imageBatch, generatedImages])

      # Labels for generated and real data
      yDis = np.zeros(2*batchSize)
      # One-sided label smoothing
      yDis[:batchSize] = 0.9

      # Train discriminator
      discriminator.trainable = True
      dloss = discriminator.train_on_batch(X, yDis)

      # Train generator
      noise = np.random.normal(0, 1, size=[batchSize, randomDim])
      yGen = np.ones(batchSize)
      discriminator.trainable = False
      gloss = gan.train_on_batch(noise, yGen)

      if e == 1 or e % 20 == 0:
        saveGeneratedImages(e)
  

Now in the same for loop, we trained the generator. We wanted the images
generated by the generator to be detected as real by the discriminator, so we used
a random vector (noise) as input to the generator; this generated a fake image
and then trained the GAN such that the discriminator perceived the image as real
(output 1):


OPTIONAL: Cool trick, right? If you wish to, you can save the generator and discriminator loss
as well as the generated images. Next, we are saving the losses for each epoch and
generating images after every 20 epochs:

To plot the loss and the generated images of the handwritten digits, we define two
helper functions, plotLoss() and saveGeneratedImages(). Their code is given
as follows:

In [37]:
# Plot the loss from each batch
def plotLoss(epoch):
  plt.figure(figsize=(10, 8))
  plt.plot(dLosses, label='Discriminitive loss')
  plt.plot(gLosses, label='Generative loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.savefig('images/gan_loss_epoch_%d.png' % epoch)

# Create a wall of generated MNIST images
def saveGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)
    plt.figure(figsize=figsize)

    for i in range(generatedImages.shape[0]):
      plt.subplot(dim[0], dim[1], i+1)
      plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
      plt.axis('off')
      plt.tight_layout()
      plt.savefig('images/gan_generated_image_epoch_%d.png' % epoch)

The complete code for this can be found in the notebook [VanillaGAN.ipynb](https://github.com/squeeko/Deep-Learning-with-TensorFlow-2-and-Keras/blob/master/Chapter%206/VanillaGAN.ipynb) at
the GitHub repo for this chapter.

In [40]:
    plotLoss(e)

NameError: ignored